# A Recommender System for Groceries Contractor

## Import Libraries

In [3]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
from bs4 import BeautifulSoup
import requests # library to handle requests
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# !conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
import geopy.geocoders # convert an address into latitude and longitude values

##!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
!pip install folium
import folium # map rendering library


print('Libraries are imported.')

     |████████████████████████████████| 102kB 7.1MB/s ta 0:00:011
Libraries are imported.


## Postal Codes in Toronto

In [14]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
read_table = pd.read_html(url,header=[0])
df1 = read_table[0]

# rename columns' name
df1 = df1.rename(index=str, columns={'Postcode':'PostalCode','Neighbourhood':'Neighborhood'})

# Ignore cells with a borough that is Not assigned
df1 = df1[df1.Borough !='Not assigned']
df1.reset_index(drop=True,inplace=True)

# groupby
df1 = df1.groupby('Postal Code',as_index=False).agg(lambda x: ','.join(set(x.dropna())))

# If a cell has a borough but a Not assigned neighborhood, 
#then the neighborhood will be the same as the borough
df1.loc[df1['Neighborhood'] == 'Not assigned','Neighborhood'] = df1['Borough']

df1.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [15]:
df2 = pd.read_csv('http://cocl.us/Geospatial_data')
df2.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [16]:
df2.columns = ['Postal Code','Latitude','Longitude']
df2.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [18]:
df_toronto = pd.merge(left=df1,right=df2,on='Postal Code')
df_toronto.head(15)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


## Create a Map of Toronto City (with its Postal Codes' Regions)

In [20]:
# for the city Toronto, latitude and longtitude are manually extracted via google search
toronto_latitude = 43.6932; toronto_longitude = -79.3832
map_toronto = folium.Map(location = [toronto_latitude, toronto_longitude], zoom_start = 10.7)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    

map_toronto

## Focusing on the "Scarorough" Borough in Toronto (its neighborhoods)

In [33]:
#df_toronto['Borough'] == 'Scarborough'

# selecting only neighborhoods regarding to "Scarborough" borough.
scarborough_data = df_toronto[df_toronto['Borough'] == 'Scarborough']
scarborough_data = scarborough_data.reset_index(drop=True)
scarborough_data.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## Create a Map of Scarborough and Its Neighbourhoods

In [34]:
address_scar = 'Scarborough, Toronto'
latitude_scar = 43.773077
longitude_scar = -79.257774
print('The geograpical coordinate of "Scarborough" are: {}, {}.'.format(latitude_scar, longitude_scar))

map_Scarborough = folium.Map(location=[latitude_scar, longitude_scar], zoom_start=11.5)

# add markers to map
for lat, lng, label in zip(scarborough_data['Latitude'], scarborough_data['Longitude'], scarborough_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius = 10,
        popup = label,
        color ='blue',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.7).add_to(map_Scarborough)  
    
map_Scarborough

The geograpical coordinate of "Scarborough" are: 43.773077, -79.257774.


In [35]:
def foursquare_crawler (postal_code_list, neighborhood_list, lat_list, lng_list, LIMIT = 500, radius = 1000):
    result_ds = []
    counter = 0
    for postal_code, neighborhood, lat, lng in zip(postal_code_list, neighborhood_list, lat_list, lng_list):
         
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID , CLIENT_SECRET,VERSION , lat, lng, radius, LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        tmp_dict = {}
        tmp_dict['Postal Code'] = postal_code; tmp_dict['Neighborhood(s)'] = neighborhood; 
        tmp_dict['Latitude'] = lat; tmp_dict['Longitude'] = lng;
        tmp_dict['Crawling_result'] = results;
        result_ds.append(tmp_dict)
        counter += 1
        print('{}.'.format(counter))
        print('Data is Obtained, for the Postal Code {} (and Neighborhoods {}) SUCCESSFULLY.'.format(postal_code, neighborhood))
    return result_ds;

In [36]:
CLIENT_ID = 'XG4LH51YMKUW42WZCAKMWJHAZK4LGILOWRWCQ0WUSZNYVNJI' # your Foursquare ID
CLIENT_SECRET = '3NSLRA14UO4LMSN1FM5RRBWVHKHCDSVT1WFMD32KHUFHRVC1' # your Foursquare Secret
VERSION = '20200824' # Foursquare API version

##  Crawling Internet (in fact only Foursquare database) for

## Venues in the Neighborhoods inside "Scarborough"

In [38]:
print('Crawling different neighborhoods inside "Scarborough"')
Scarborough_foursquare_dataset = foursquare_crawler(list(scarborough_data['Postal Code']),
                                                   list(scarborough_data['Neighborhood']),
                                                   list(scarborough_data['Latitude']),
                                                   list(scarborough_data['Longitude']),)

Crawling different neighborhoods inside "Scarborough"
1.
Data is Obtained, for the Postal Code M1B (and Neighborhoods Malvern, Rouge) SUCCESSFULLY.
2.
Data is Obtained, for the Postal Code M1C (and Neighborhoods Rouge Hill, Port Union, Highland Creek) SUCCESSFULLY.
3.
Data is Obtained, for the Postal Code M1E (and Neighborhoods Guildwood, Morningside, West Hill) SUCCESSFULLY.
4.
Data is Obtained, for the Postal Code M1G (and Neighborhoods Woburn) SUCCESSFULLY.
5.
Data is Obtained, for the Postal Code M1H (and Neighborhoods Cedarbrae) SUCCESSFULLY.
6.
Data is Obtained, for the Postal Code M1J (and Neighborhoods Scarborough Village) SUCCESSFULLY.
7.
Data is Obtained, for the Postal Code M1K (and Neighborhoods Kennedy Park, Ionview, East Birchmount Park) SUCCESSFULLY.
8.
Data is Obtained, for the Postal Code M1L (and Neighborhoods Golden Mile, Clairlea, Oakridge) SUCCESSFULLY.
9.
Data is Obtained, for the Postal Code M1M (and Neighborhoods Cliffside, Cliffcrest, Scarborough Village West) 

## Breakpoint:

## Saving results of Foursquare, so that we would not need to connect every time to Foursquare (and use our portions) .

In [39]:
import pickle
with open("Scarborough_foursquare_dataset.txt", "wb") as fp:   #Pickling
    pickle.dump(Scarborough_foursquare_dataset, fp)
print('Received Data from Internet is Saved to Computer.')

Received Data from Internet is Saved to Computer.


In [41]:
with open("Scarborough_foursquare_dataset.txt", "rb") as fp:   # Unpickling
    Scarborough_foursquare_dataset = pickle.load(fp)
    print(type(Scarborough_foursquare_dataset))
Scarborough_foursquare_dataset


<class 'list'>


[{'Postal Code': 'M1B',
  'Neighborhood(s)': 'Malvern, Rouge',
  'Latitude': 43.806686299999996,
  'Longitude': -79.19435340000001,
  'Crawling_result': [{'reasons': {'count': 0,
     'items': [{'summary': 'This spot is popular',
       'type': 'general',
       'reasonName': 'globalInteractionReason'}]},
    'venue': {'id': '4d669cba83865481c948fa53',
     'name': 'Images Salon & Spa',
     'location': {'address': '8130 Sheppard Ave E',
      'crossStreet': 'Morningside Ave',
      'lat': 43.80228301948931,
      'lng': -79.19856472801668,
      'labeledLatLngs': [{'label': 'display',
        'lat': 43.80228301948931,
        'lng': -79.19856472801668}],
      'distance': 595,
      'postalCode': 'M1B 3W3',
      'cc': 'CA',
      'city': 'Toronto',
      'state': 'ON',
      'country': 'Canada',
      'formattedAddress': ['8130 Sheppard Ave E (Morningside Ave)',
       'Toronto ON M1B 3W3',
       'Canada']},
     'categories': [{'id': '4bf58dd8d48988d1ed941735',
       'name': 'Spa'

## Cleaning the RAW Data Received from Foursquare Database

In [44]:
# This function is created to connect to the saved list which is the received database. It will extract each venue 
# for every neighborhood inside the database

def get_venue_dataset(foursquare_dataset):
    result_df = pd.DataFrame(columns = ['Postal Code', 'Neighborhood', 
                                           'Neighborhood Latitude', 'Neighborhood Longitude',
                                          'Venue', 'Venue Summary', 'Venue Category', 'Distance'])
    # print(result_df)
    
    for neigh_dict in foursquare_dataset:
        postal_code = neigh_dict['Postal Code']; neigh = neigh_dict['Neighborhood(s)']
        lat = neigh_dict['Latitude']; lng = neigh_dict['Longitude']
        print('Number of Venuse in Coordination "{}" Posal Code and "{}" Negihborhood(s) is:'.format(postal_code, neigh))
        print(len(neigh_dict['Crawling_result']))
        
        for venue_dict in neigh_dict['Crawling_result']:
            summary = venue_dict['reasons']['items'][0]['summary']
            name = venue_dict['venue']['name']
            dist = venue_dict['venue']['location']['distance']
            cat =  venue_dict['venue']['categories'][0]['name']
            
            
            # print({'Postal Code': postal_code, 'Neighborhood': neigh, 
            #                   'Neighborhood Latitude': lat, 'Neighborhood Longitude':lng,
            #                   'Venue': name, 'Venue Summary': summary, 
            #                   'Venue Category': cat, 'Distance': dist})
            
            result_df = result_df.append({'Postal Code': postal_code, 'Neighborhood': neigh, 
                              'Neighborhood Latitude': lat, 'Neighborhood Longitude':lng,
                              'Venue': name, 'Venue Summary': summary, 
                              'Venue Category': cat, 'Distance': dist}, ignore_index = True)
            # print(result_df)
    
    return(result_df)

In [45]:

scarborough_venues = get_venue_dataset(Scarborough_foursquare_dataset)

Number of Venuse in Coordination "M1B" Posal Code and "Malvern, Rouge" Negihborhood(s) is:
17
Number of Venuse in Coordination "M1C" Posal Code and "Rouge Hill, Port Union, Highland Creek" Negihborhood(s) is:
5
Number of Venuse in Coordination "M1E" Posal Code and "Guildwood, Morningside, West Hill" Negihborhood(s) is:
25
Number of Venuse in Coordination "M1G" Posal Code and "Woburn" Negihborhood(s) is:
8
Number of Venuse in Coordination "M1H" Posal Code and "Cedarbrae" Negihborhood(s) is:
31
Number of Venuse in Coordination "M1J" Posal Code and "Scarborough Village" Negihborhood(s) is:
12
Number of Venuse in Coordination "M1K" Posal Code and "Kennedy Park, Ionview, East Birchmount Park" Negihborhood(s) is:
28
Number of Venuse in Coordination "M1L" Posal Code and "Golden Mile, Clairlea, Oakridge" Negihborhood(s) is:
30
Number of Venuse in Coordination "M1M" Posal Code and "Cliffside, Cliffcrest, Scarborough Village West" Negihborhood(s) is:
12
Number of Venuse in Coordination "M1N" Pos

## Showing Venues for Each Neighborhood in Scarborough

In [46]:
scarborough_venues.head()

,Postal Code,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Summary,Venue Category,Distance
0,M1B,"Malvern, Rouge",43.806686,-79.194353,Images Salon & Spa,This spot is popular,Spa,595
1,M1B,"Malvern, Rouge",43.806686,-79.194353,Harvey's,This spot is popular,Restaurant,807
2,M1B,"Malvern, Rouge",43.806686,-79.194353,Staples Morningside,This spot is popular,Paper / Office Supplies Store,735
3,M1B,"Malvern, Rouge",43.806686,-79.194353,Wendy's,This spot is popular,Fast Food Restaurant,600
4,M1B,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,This spot is popular,Fast Food Restaurant,387


In [47]:
scarborough_venues.tail()

,Postal Code,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Summary,Venue Category,Distance
390,M1W,"Steeles West, L'Amoreaux West",43.799525,-79.318389,Divine Wok Restaurant,This spot is popular,Chinese Restaurant,957
391,M1W,"Steeles West, L'Amoreaux West",43.799525,-79.318389,Buddy Cafe,This spot is popular,Chinese Restaurant,973
392,M1W,"Steeles West, L'Amoreaux West",43.799525,-79.318389,Birchwood Plaza,This spot is popular,Shopping Mall,977
393,M1W,"Steeles West, L'Amoreaux West",43.799525,-79.318389,Olympian Swimming,This spot is popular,Gym Pool,978
394,M1W,"Steeles West, L'Amoreaux West",43.799525,-79.318389,Dumpling & Szechuan Cuisine（川流不息店）,This spot is popular,Chinese Restaurant,989


## Breakpoint:

### End of Processing the Retrieved Information from Foursquare
### Saving a Cleaned Version of DataFrame as the Results from Foursquare

In [48]:
scarborough_venues.to_csv('scarborough_venues.csv')

In [49]:
## Loading Data from File (Saved "Foursquare " DataFrame for Venues)
scarborough_venues = pd.read_csv('scarborough_venues.csv')

## Some Summary Information about Neighborhoods inside "Scarborough"

In [50]:
neigh_list = list(scarborough_venues['Neighborhood'].unique())
print('Number of Neighborhoods inside Scarborough:')
print(len(neigh_list))
print('List of Neighborhoods inside Scarborough:')
neigh_list

Number of Neighborhoods inside Scarborough:
16
List of Neighborhoods inside Scarborough:


['Malvern, Rouge',
 'Rouge Hill, Port Union, Highland Creek',
 'Guildwood, Morningside, West Hill',
 'Woburn',
 'Cedarbrae',
 'Scarborough Village',
 'Kennedy Park, Ionview, East Birchmount Park',
 'Golden Mile, Clairlea, Oakridge',
 'Cliffside, Cliffcrest, Scarborough Village West',
 'Birch Cliff, Cliffside West',
 'Dorset Park, Wexford Heights, Scarborough Town Centre',
 'Wexford, Maryvale',
 'Agincourt',
 "Clarks Corners, Tam O'Shanter, Sullivan",
 "Milliken, Agincourt North, Steeles East, L'Amoreaux East",
 "Steeles West, L'Amoreaux West"]

## Some Summary Information about Neighborhoods inside "Scarborough" Cont'd

In [51]:
neigh_venue_summary = scarborough_venues.groupby('Neighborhood').count()
neigh_venue_summary.drop(columns = ['Unnamed: 0']).head()

,Postal Code,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Summary,Venue Category,Distance
Neighborhood,,,,,,,
Agincourt,42,42,42,42,42,42,42
"Birch Cliff, Cliffside West",14,14,14,14,14,14,14
Cedarbrae,31,31,31,31,31,31,31
"Clarks Corners, Tam O'Shanter, Sullivan",36,36,36,36,36,36,36
"Cliffside, Cliffcrest, Scarborough Village West",12,12,12,12,12,12,12


In [52]:
print('There are {} uniques categories.'.format(len(scarborough_venues['Venue Category'].unique())))

print('Here is the list of different categories:')
list(scarborough_venues['Venue Category'].unique())

There are 110 uniques categories.
Here is the list of different categories:


['Spa',
 'Restaurant',
 'Paper / Office Supplies Store',
 'Fast Food Restaurant',
 'Bank',
 'Caribbean Restaurant',
 'Coffee Shop',
 'Hardware Store',
 'Hobby Shop',
 'Trail',
 'Chinese Restaurant',
 'Gym',
 'Supermarket',
 'Sandwich Place',
 'Burger Joint',
 'Italian Restaurant',
 'Breakfast Spot',
 'Playground',
 'Park',
 'Fried Chicken Joint',
 'Liquor Store',
 'Pizza Place',
 'Food & Drink Shop',
 'Smoothie Shop',
 'Beer Store',
 'Pharmacy',
 'Sports Bar',
 'Discount Store',
 'Greek Restaurant',
 'Rental Car Location',
 'Convenience Store',
 'Video Game Store',
 'Shopping Mall',
 'Indian Restaurant',
 'Mobile Phone Shop',
 'Hakka Restaurant',
 'Thai Restaurant',
 'Music Store',
 'Athletics & Sports',
 'Bakery',
 'Gas Station',
 'Yoga Studio',
 'Grocery Store',
 'Wings Joint',
 'Lounge',
 'Asian Restaurant',
 'Bus Line',
 'Sporting Goods Shop',
 'Ice Cream Shop',
 'Train Station',
 'Japanese Restaurant',
 'Bowling Alley',
 'Department Store',
 'Bus Station',
 'Metro Station',
 'Ligh

In [53]:
# Just for fun and deeper understanding
print(type(scarborough_venues[['Venue Category']]))

print(type(scarborough_venues['Venue Category']))

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.series.Series'>


## One-hot Encoding the "categroies" Column into Every Unique Categorical Feature.

In [54]:
# one hot encoding
scarborough_onehot = pd.get_dummies(data = scarborough_venues, drop_first  = False, 
                              prefix = "", prefix_sep = "", columns = ['Venue Category'])
scarborough_onehot.head()

,Unnamed: 0,Postal Code,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Summary,Distance,Accessories Store,African Restaurant,American Restaurant,Asian Restaurant,Athletics & Sports,Auto Garage,Automotive Shop,BBQ Joint,Badminton Court,Bakery,Bank,Beach,Beer Store,Bowling Alley,Breakfast Spot,Bubble Tea Shop,Burger Joint,Bus Line,Bus Station,Café,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,College Stadium,Convenience Store,Department Store,Dessert Shop,Diner,Discount Store,Electronics Store,Event Space,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Food & Drink Shop,Food Court,Fried Chicken Joint,Furniture / Home Store,Gas Station,General Entertainment,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Hardware Store,Hobby Shop,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Malay Restaurant,Market,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Music Store,Noodle House,Other Great Outdoors,Paper / Office Supplies Store,Park,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pool,Pool Hall,Pub,Rental Car Location,Restaurant,Sandwich Place,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Soccer Field,Spa,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Supermarket,Sushi Restaurant,Taiwanese Restaurant,Thai Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Shop,Wings Joint,Yoga Studio
0,0,M1B,"Malvern, Rouge",43.806686,-79.194353,Images Salon & Spa,This spot is popular,595,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,M1B,"Malvern, Rouge",43.806686,-79.194353,Harvey's,This spot is popular,807,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,M1B,"Malvern, Rouge",43.806686,-79.194353,Staples Morningside,This spot is popular,735,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,M1B,"Malvern, Rouge",43.806686,-79.194353,Wendy's,This spot is popular,600,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4,M1B,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,This spot is popular,387,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## Manually Selecting (Subsetting) Related Features for the Groceries Contractor

In [57]:
# This list is created manually 
important_list_of_features = [
 
 'Neighborhood',
 'Neighborhood Latitude',
 'Neighborhood Longitude',

 'African Restaurant',
 'American Restaurant',
 'Asian Restaurant',

 
 'BBQ Joint',
 
 'Bakery',
 
 
 
 
 
 'Breakfast Spot',

 'Burger Joint',
 
 
 
 'Cantonese Restaurant',
 'Caribbean Restaurant',
 'Chinese Restaurant',
 
 'Diner',


 'Fast Food Restaurant',
 'Filipino Restaurant',
 'Fish Market',
 'Food & Drink Shop',
 'Fried Chicken Joint',

 
 'Greek Restaurant',
 'Grocery Store',
 
 'Hakka Restaurant',

 'Hotpot Restaurant',
 
 'Indian Restaurant',

 'Italian Restaurant',
 'Japanese Restaurant',
 'Korean Restaurant',
 'Latin American Restaurant',



 'Malay Restaurant',
 
 'Mediterranean Restaurant',
 
 'Mexican Restaurant',
 'Middle Eastern Restaurant',
 
 'Noodle House',
 
 'Pizza Place',
 
 'Restaurant',
 'Sandwich Place',
 'Seafood Restaurant',
 
 'Sushi Restaurant',
 'Taiwanese Restaurant',
 
 'Thai Restaurant',
 
 'Vegetarian / Vegan Restaurant',
 
 'Vietnamese Restaurant',
 'Wings Joint']

## Updating the One-hot Encoded DataFrame and
## Grouping the Data by Neighborhoods

In [58]:
scarborough_onehot = scarborough_onehot[important_list_of_features].drop(
    columns = ['Neighborhood Latitude', 'Neighborhood Longitude']).groupby(
    'Neighborhood').sum()


scarborough_onehot.head()

,African Restaurant,American Restaurant,Asian Restaurant,BBQ Joint,Bakery,Breakfast Spot,Burger Joint,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Diner,Fast Food Restaurant,Filipino Restaurant,Fish Market,Food & Drink Shop,Fried Chicken Joint,Greek Restaurant,Grocery Store,Hakka Restaurant,Hotpot Restaurant,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Latin American Restaurant,Malay Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Noodle House,Pizza Place,Restaurant,Sandwich Place,Seafood Restaurant,Sushi Restaurant,Taiwanese Restaurant,Thai Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wings Joint
Neighborhood,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Agincourt,0,0,1,0,2,1,0,1,2,6,0,0,1,0,0,0,0,1,0,0,1,0,0,0,1,1,1,0,0,1,1,1,2,1,1,0,0,0,1,0
"Birch Cliff, Cliffside West",0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0
Cedarbrae,0,0,1,0,3,0,1,0,1,1,0,1,0,0,0,1,0,1,1,0,2,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1
"Clarks Corners, Tam O'Shanter, Sullivan",0,0,0,0,1,0,0,1,1,1,0,3,0,0,0,1,1,1,0,0,0,1,0,0,0,0,0,1,0,1,1,0,2,1,0,1,1,0,1,0
"Cliffside, Cliffcrest, Scarborough Village West",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,1,0,0,0,0,0,0,0,0


## Integrating Different Restaurants and Different Joints
(Assuming Different Resaturants Use the Same Raw Groceries)
This Assumption is made for simplicity and due to not having very large dataset about neighborhoods

In [59]:
feat_name_list = list(scarborough_onehot.columns)
restaurant_list = []


for counter, value in enumerate(feat_name_list):
    if value.find('Restaurant') != (-1):
        restaurant_list.append(value)
        
scarborough_onehot['Total Restaurants'] = scarborough_onehot[restaurant_list].sum(axis = 1)
scarborough_onehot = scarborough_onehot.drop(columns = restaurant_list)


feat_name_list = list(scarborough_onehot.columns)
joint_list = []


for counter, value in enumerate(feat_name_list):
    if value.find('Joint') != (-1):
        joint_list.append(value)
        
scarborough_onehot['Total Joints'] = scarborough_onehot[joint_list].sum(axis = 1)
scarborough_onehot = scarborough_onehot.drop(columns = joint_list)

## Showing the Fully-Processed DataFrame about Neighborhoods inside Scarborrough.
## This Dataset is Ready for any Machine Learning Algorithm.

In [60]:
scarborough_onehot

,Bakery,Breakfast Spot,Diner,Fish Market,Food & Drink Shop,Grocery Store,Noodle House,Pizza Place,Sandwich Place,Total Restaurants,Total Joints
Neighborhood,,,,,,,,,,,
Agincourt,2,1,0,0,0,1,1,1,2,19,0
"Birch Cliff, Cliffside West",0,0,1,0,0,0,0,0,0,2,0
Cedarbrae,3,0,0,0,0,1,0,1,0,8,3
"Clarks Corners, Tam O'Shanter, Sullivan",1,0,0,0,0,1,1,1,2,13,1
"Cliffside, Cliffcrest, Scarborough Village West",0,0,0,0,0,0,0,3,0,1,0
"Dorset Park, Wexford Heights, Scarborough Town Centre",1,0,0,0,0,0,0,1,1,14,2
"Golden Mile, Clairlea, Oakridge",2,0,1,0,0,1,0,1,1,3,0
"Guildwood, Morningside, West Hill",0,0,0,0,1,0,0,3,1,4,1
"Kennedy Park, Ionview, East Birchmount Park",0,0,0,0,0,2,0,2,1,6,1


## Run k-means to Cluster Neighborhoods into 5 Clusters

In [62]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# run k-means clustering
kmeans = KMeans(n_clusters = 5, random_state = 0).fit(scarborough_onehot)

## Showing Centers of Each Cluster

In [63]:
means_df = pd.DataFrame(kmeans.cluster_centers_)
means_df.columns = scarborough_onehot.columns
means_df.index = ['G1','G2','G3','G4','G5']
means_df['Total Sum'] = means_df.sum(axis = 1)
means_df.sort_values(axis = 0, by = ['Total Sum'], ascending=False)

,Bakery,Breakfast Spot,Diner,Fish Market,Food & Drink Shop,Grocery Store,Noodle House,Pizza Place,Sandwich Place,Total Restaurants,Total Joints,Total Sum
G3,2.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,2.000000,19.000000,0.000000,27.000000
G5,1.000000,0.000000,0.000000,0.000000,0.000000,0.666667,0.666667,1.333333,1.000000,13.000000,1.666667,19.333333
G2,2.000000,0.666667,0.000000,0.333333,0.000000,1.666667,0.000000,2.000000,0.333333,9.666667,1.666667,18.333333
G4,0.000000,0.000000,0.000000,0.000000,0.333333,0.666667,0.000000,1.666667,1.000000,5.000000,0.666667,9.333333
G1,0.333333,0.166667,0.333333,0.000000,0.000000,0.333333,0.000000,0.833333,0.333333,2.166667,0.166667,4.666667


## Result:
### Best Group is G3;
### Second Best Group is G5;
### Third Best Group is G2;
Inserting "kmeans.labels_" into the Original Scarborough DataFrame

## Finding the Corresponding Group for Each Neighborhood.

In [65]:
neigh_summary = pd.DataFrame([scarborough_onehot.index, 1 + kmeans.labels_]).T
neigh_summary.columns = ['Neighborhood', 'Group']
neigh_summary

,Neighborhood,Group
0,Agincourt,3
1,"Birch Cliff, Cliffside West",1
2,Cedarbrae,2
3,"Clarks Corners, Tam O'Shanter, Sullivan",5
4,"Cliffside, Cliffcrest, Scarborough Village West",1
5,"Dorset Park, Wexford Heights, Scarborough Town...",5
6,"Golden Mile, Clairlea, Oakridge",1
7,"Guildwood, Morningside, West Hill",4
8,"Kennedy Park, Ionview, East Birchmount Park",4
9,"Malvern, Rouge",4


## Deducing Results:
## Best Neighborhood Are...

In [68]:
neigh_summary[neigh_summary['Group'] == 3]

,Neighborhood,Group
0,Agincourt,3


In [69]:
name_of_neigh = list(neigh_summary[neigh_summary['Group'] == 3]['Neighborhood'])[0]
scarborough_venues[scarborough_venues['Neighborhood'] == name_of_neigh].iloc[0,1:5].to_dict()


{'Postal Code': 'M1S',
 'Neighborhood': 'Agincourt',
 'Neighborhood Latitude': 43.7942003,
 'Neighborhood Longitude': -79.26202940000002}

## Second Best Neighborhoods

In [70]:
neigh_summary[neigh_summary['Group'] == 5]

,Neighborhood,Group
3,"Clarks Corners, Tam O'Shanter, Sullivan",5
5,"Dorset Park, Wexford Heights, Scarborough Town...",5
10,"Milliken, Agincourt North, Steeles East, L'Amo...",5


## Third Best Neighborhood

In [71]:
neigh_summary[neigh_summary['Group'] == 2]

,Neighborhood,Group
2,Cedarbrae,2
13,"Steeles West, L'Amoreaux West",2
14,"Wexford, Maryvale",2


## Thank You!